In [1]:
import pandas as pd

In [2]:
data = {
        'tra': pd.read_csv('./input/train.csv',
                           index_col=0,
                           header=0,
                           names=['obs_id', 'meter_id', 'Timestamp', 'Values'],
                           parse_dates=['Timestamp'],
                           low_memory=False),
        'meta': pd.read_csv('./input/metadata.csv',
                            index_col=False,
                            low_memory=False),
        'hol': pd.read_csv('./input/holidays.csv',
                           index_col=0,
                           parse_dates=['Date'],
                           low_memory=False),
        'wx': pd.read_csv('./input/weather.csv',
                          index_col=0,
                          header=0,
                          names=['wx_id', 'Timestamp', 'Temperature', 'Distance', 'site_id'],
                          parse_dates=['Timestamp'],
                          low_memory=False)
    }

# Date object creation and formatting
for df in ['tra', 'wx']:
    data[df]['Date'] = data[df]['Timestamp'].dt.date
data['hol']['Date'] = data['hol']['Date'].dt.date

D:\Users\Luis\Programs\Anaconda3\envs\PREDICT-env\lib\site-packages\numpy\lib\arraysetops.py:395: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
data['tra'].shape

(43668606, 4)

In [4]:
data['meta'].shape

(187, 6)

In [5]:
data['hol'].shape

(234, 3)

In [6]:
data['wx'].shape

(391628, 5)

In [7]:
data['tra'].head()

,meter_id,Timestamp,Values,Date
obs_id,,,,
2532,2,2015-06-11 00:00:00,2035.0,2015-06-11
2543,2,2015-06-11 00:15:00,2074.0,2015-06-11
2544,2,2015-06-11 00:30:00,2062.0,2015-06-11
2525,2,2015-06-11 00:45:00,2025.0,2015-06-11
2534,2,2015-06-11 01:00:00,2034.0,2015-06-11


In [8]:
data['meta'].head()

,meter_id,site_id,meter_description,units,surface,activity
0,234_203,234_203,virtual main,Wh,5750.0,office
1,863,234_203,main meter,Wh,5750.0,office
2,869,234_203,other,Wh,5750.0,office
3,872,234_203,elevators,Wh,5750.0,office
4,875,234_203,Lighting,Wh,5750.0,office


In [9]:
# Join datasets
final = pd.merge(data['tra'], data['meta'], how='left', on=['meter_id'])

In [10]:
final.shape

(43668606, 9)

In [11]:
final.head()

,meter_id,Timestamp,Values,Date,site_id,meter_description,units,surface,activity
0,2,2015-06-11 00:00:00,2035.0,2015-06-11,334_61,main meter,Wh,2000.0,office
1,2,2015-06-11 00:15:00,2074.0,2015-06-11,334_61,main meter,Wh,2000.0,office
2,2,2015-06-11 00:30:00,2062.0,2015-06-11,334_61,main meter,Wh,2000.0,office
3,2,2015-06-11 00:45:00,2025.0,2015-06-11,334_61,main meter,Wh,2000.0,office
4,2,2015-06-11 01:00:00,2034.0,2015-06-11,334_61,main meter,Wh,2000.0,office


In [19]:
data['hol'].head()

,Date,Holiday,site_id
row_id,,,
0,2016-01-01,New year,038
1,2016-03-28,Easter Monday,038
2,2016-05-01,Labour Day,038
3,2016-05-05,Ascension Thursday,038
4,2016-05-08,Victory in Europe Day,038


It isn't evident from a cursory look at the data, but there are duplicate holidays with composite key = Date, site_id. 

In order to left join the holiday data without creating duplicate records, we dummy code Holiday to = 1 and drop any duplicates

In [12]:
data['hol'].Holiday = 1

In [13]:
data['hol'] = data['hol'].drop_duplicates()

In [14]:
final = pd.merge(final, data['hol'], how='left', on=['Date', 'site_id'])

In [15]:
final.shape

(43668606, 10)

In [16]:
final.head()

,meter_id,Timestamp,Values,Date,site_id,meter_description,units,surface,activity,Holiday
0,2,2015-06-11 00:00:00,2035.0,2015-06-11,334_61,main meter,Wh,2000.0,office,NaN
1,2,2015-06-11 00:15:00,2074.0,2015-06-11,334_61,main meter,Wh,2000.0,office,NaN
2,2,2015-06-11 00:30:00,2062.0,2015-06-11,334_61,main meter,Wh,2000.0,office,NaN
3,2,2015-06-11 00:45:00,2025.0,2015-06-11,334_61,main meter,Wh,2000.0,office,NaN
4,2,2015-06-11 01:00:00,2034.0,2015-06-11,334_61,main meter,Wh,2000.0,office,NaN


In [17]:
# Fill NaNs in Holiday with 0

final['Holiday'].fillna(value=0, inplace=True)

In [18]:
final.head()

,meter_id,Timestamp,Values,Date,site_id,meter_description,units,surface,activity,Holiday
0,2,2015-06-11 00:00:00,2035.0,2015-06-11,334_61,main meter,Wh,2000.0,office,0.0
1,2,2015-06-11 00:15:00,2074.0,2015-06-11,334_61,main meter,Wh,2000.0,office,0.0
2,2,2015-06-11 00:30:00,2062.0,2015-06-11,334_61,main meter,Wh,2000.0,office,0.0
3,2,2015-06-11 00:45:00,2025.0,2015-06-11,334_61,main meter,Wh,2000.0,office,0.0
4,2,2015-06-11 01:00:00,2034.0,2015-06-11,334_61,main meter,Wh,2000.0,office,0.0


In [19]:
data['wx'].head()

,Timestamp,Temperature,Distance,site_id,Date
wx_id,,,,,
42628,2012-01-01 01:00:00,3.9,11.902932,38,2012-01-01
42629,2012-01-01 02:00:00,4.1,11.902932,38,2012-01-01
42630,2012-01-01 03:00:00,4.2,11.902932,38,2012-01-01
42631,2012-01-01 04:00:00,4.1,11.902932,38,2012-01-01
42632,2012-01-01 05:00:00,4.3,11.902932,38,2012-01-01


In [21]:
sum(data['wx'].duplicated())

18

In [24]:
data['wx'].groupby(data['wx'].columns.tolist(), as_index=False).size().sort_values(ascending=False)

Timestamp            Temperature  Distance   site_id  Date      
2016-10-30 03:20:00  9.0          17.159545  334_61   2016-10-30    2
2014-10-26 03:50:00  14.0         19.734144  334_61   2014-10-26    2
2017-10-29 03:50:00  14.0         17.159545  334_61   2017-10-29    2
2017-10-29 03:20:00  14.0         19.734144  334_61   2017-10-29    2
                                  17.159545  334_61   2017-10-29    2
2016-10-30 03:20:00  13.0         19.734144  334_61   2016-10-30    2
2014-10-17 12:20:00  27.0         19.734144  334_61   2014-10-17    2
2014-10-17 11:50:00  27.0         19.734144  334_61   2014-10-17    2
2014-10-26 03:50:00  15.0         17.159545  334_61   2014-10-26    2
2012-10-28 03:20:00  22.0         17.159545  334_61   2012-10-28    2
2012-10-28 03:50:00  22.0         17.159545  334_61   2012-10-28    2
                     23.0         19.734144  334_61   2012-10-28    2
2014-10-26 03:20:00  14.0         19.734144  334_61   2014-10-26    2
2015-04-05 02:30:00  16.0

In [25]:
data['wx'] = data['wx'].drop_duplicates()

In [26]:
data['wx'].groupby(data['wx'].columns.tolist(), as_index=False).size().sort_values(ascending=False)

Timestamp            Temperature  Distance   site_id  Date      
2017-12-31 10:30:00  21.0         10.125819  234_203  2017-12-31    1
2014-04-26 08:20:00  16.0         17.159545  334_61   2014-04-26    1
2014-04-26 07:00:00  11.5         11.902932  38       2014-04-26    1
                     16.0         10.125819  234_203  2014-04-26    1
2014-04-26 07:20:00  13.0         17.159545  334_61   2014-04-26    1
                     15.0         19.734144  334_61   2014-04-26    1
2014-04-26 07:30:00  16.0         10.125819  234_203  2014-04-26    1
2014-04-26 07:50:00  15.0         17.159545  334_61   2014-04-26    1
                     16.0         19.734144  334_61   2014-04-26    1
2014-04-26 08:00:00  11.7         11.902932  38       2014-04-26    1
                     16.0         10.125819  234_203  2014-04-26    1
2014-04-26 08:20:00  17.0         19.734144  334_61   2014-04-26    1
2014-04-26 06:50:00  13.0         17.159545  334_61   2014-04-26    1
2014-04-26 08:30:00  16.0

In [27]:
data['wx'].groupby(['Timestamp', 'site_id'], as_index=False).size().sort_values(ascending=False)

Timestamp            site_id
2015-10-25 03:20:00  334_61     4
2015-12-21 14:00:00  334_61     4
2015-08-10 03:00:00  334_61     3
2015-10-13 12:00:00  334_61     3
2015-01-22 05:00:00  334_61     3
2015-04-11 15:00:00  334_61     3
2016-03-18 11:00:00  334_61     3
2017-06-14 18:00:00  334_61     3
2017-01-16 14:00:00  334_61     3
2015-08-10 00:00:00  334_61     3
2017-10-29 14:00:00  334_61     3
2016-09-28 18:00:00  334_61     3
2017-04-17 21:00:00  334_61     3
2015-04-11 12:00:00  334_61     3
2016-03-18 08:00:00  334_61     3
2015-12-24 23:00:00  334_61     3
2017-08-20 09:00:00  334_61     3
2016-07-24 09:00:00  334_61     3
2016-11-26 05:00:00  334_61     3
2017-04-17 18:00:00  334_61     3
2015-12-24 20:00:00  334_61     3
2016-09-28 21:00:00  334_61     3
2017-04-17 15:00:00  334_61     3
2014-10-23 03:00:00  334_61     3
2015-08-09 18:00:00  334_61     3
2016-07-24 00:00:00  334_61     3
2017-06-14 21:00:00  334_61     3
2015-04-11 21:00:00  334_61     3
2016-03-18 17:00:00

In [30]:
data['wx'][(data['wx']['Timestamp'] == '2015-10-25 03:20:00') & (data['wx']['site_id'] == '334_61')].head()

,Timestamp,Temperature,Distance,site_id,Date
wx_id,,,,,
101611,2015-10-25 03:20:00,15.0,19.734144,334_61,2015-10-25
101613,2015-10-25 03:20:00,16.0,19.734144,334_61,2015-10-25
119492,2015-10-25 03:20:00,15.0,17.159545,334_61,2015-10-25
119494,2015-10-25 03:20:00,14.0,17.159545,334_61,2015-10-25


In [31]:
agg = data['wx'].groupby(['Timestamp', 'site_id']).mean()

In [33]:
agg.reset_index().head()

,Timestamp,site_id,Temperature,Distance
0,2012-01-01 01:00:00,38,3.9,11.902932
1,2012-01-01 02:00:00,38,4.1,11.902932
2,2012-01-01 02:20:00,334_61,8.5,18.446845
3,2012-01-01 02:50:00,334_61,8.0,18.446845
4,2012-01-01 03:00:00,38,4.2,11.902932


In [34]:
final = pd.merge(final, agg.reset_index(), how='left', on=['Timestamp', 'site_id'])

In [35]:
final.shape

(43668606, 12)

In [36]:
final.head()

,meter_id,Timestamp,Values,Date,site_id,meter_description,units,surface,activity,Holiday,Temperature,Distance
0,2,2015-06-11 00:00:00,2035.0,2015-06-11,334_61,main meter,Wh,2000.0,office,0.0,20.033333,16.317674
1,2,2015-06-11 00:15:00,2074.0,2015-06-11,334_61,main meter,Wh,2000.0,office,0.0,NaN,NaN
2,2,2015-06-11 00:30:00,2062.0,2015-06-11,334_61,main meter,Wh,2000.0,office,0.0,NaN,NaN
3,2,2015-06-11 00:45:00,2025.0,2015-06-11,334_61,main meter,Wh,2000.0,office,0.0,NaN,NaN
4,2,2015-06-11 01:00:00,2034.0,2015-06-11,334_61,main meter,Wh,2000.0,office,0.0,NaN,NaN


In [37]:
final = final.rename(str.lower, axis='columns')

In [38]:
final.shape

(43668606, 12)

In [39]:
final.head()

,meter_id,timestamp,values,date,site_id,meter_description,units,surface,activity,holiday,temperature,distance
0,2,2015-06-11 00:00:00,2035.0,2015-06-11,334_61,main meter,Wh,2000.0,office,0.0,20.033333,16.317674
1,2,2015-06-11 00:15:00,2074.0,2015-06-11,334_61,main meter,Wh,2000.0,office,0.0,NaN,NaN
2,2,2015-06-11 00:30:00,2062.0,2015-06-11,334_61,main meter,Wh,2000.0,office,0.0,NaN,NaN
3,2,2015-06-11 00:45:00,2025.0,2015-06-11,334_61,main meter,Wh,2000.0,office,0.0,NaN,NaN
4,2,2015-06-11 01:00:00,2034.0,2015-06-11,334_61,main meter,Wh,2000.0,office,0.0,NaN,NaN
